# PyTorch Implementation of the NRMS Model for `EBNeRD` (RecSys'24 Challenge)
### Course: `02456 Deep Learning` (Fall 2024)  
**Institution:** Technical University of Denmark (DTU)  
**Authors:** Kevin Moore (s204462) and Nico Tananow (s[insert number])

### Acknowledgments  
1. Special thanks to **Johannes Kruse** for his [TensorFlow implementation of the NRMS Model](https://github.com/ebanalyse/ebnerd-benchmark), which greatly supported the development of this PyTorch implementation for the EBNeRD project.  


2. Our implementation is based on the NRMS model described in the paper **["Neural News Recommendation with Multi-Head Self-Attention"](https://aclanthology.org/D19-1671/)** by Wu et al. (2019).

# Implementation

## 1. Importing Dependencies
Import all necessary libraries and modules, including `utils.model` and `utils.helper` for the NRMS model, data preparation, training, and evaluation.

In [1]:
import sys
import os
import polars as pl
os.environ["TOKENIZERS_PARALLELISM"] = "false" # to avoid warnings in transformers
from pathlib import Path
# Get the current directory
current_dir = os.getcwd()
root_dir = os.path.join(os.path.dirname(os.path.dirname(current_dir)), "dtu-02456-deep-learning-ebnerd")
src_dir = os.path.join(root_dir, "src")
# Append the relative path to the utils folder and ebrec src
sys.path.append(os.path.join(current_dir, "utils"))
sys.path.append(src_dir)
from importlib import reload

import torch
import utils.model
reload(utils.model)
from utils.model import (
    NRMSModel
)

import utils.helper
reload(utils.helper)
from utils.helper import (
		SEED,
    HParams,
    load_articles_and_embeddings,
    prepare_training_data,
    train_model,
    predict_scores,
    save_hparams
)
import utils.evaluation
reload(utils.evaluation)
from utils.evaluation import (
		save_ranked_scores,
    prepare_test_data,
    prepare_validation_data,
)

from ebrec.evaluation import MetricEvaluator, AucScore, NdcgScore, MrrScore
from ebrec.utils._python import write_submission_file
from ebrec.utils._constants import DEFAULT_IMPRESSION_ID_COL, DEFAULT_LABELS_COL

## 2. Setting Hyperparameters
Initialize the hyperparameters for the model

In [2]:
# Setting hyperparameters
hparams = HParams()
hparams.data_fraction = 1
hparams.batch_size = 32
hparams.datasplit = "ebnerd_small"

## 3. Loading and creating **embeddings** for articles
- Articles loaded from `DATASPLIT` directory 

In [3]:
DATASPLIT = hparams.datasplit
PATH = Path(os.path.join(current_dir, "data"))
print("Loading data from ", PATH, "with datasplit:", DATASPLIT)
DATASPLIT_PATH = Path(os.path.join(PATH, DATASPLIT))
# Loading articles and embeddings
article_mapping, word_embeddings = load_articles_and_embeddings(hparams, DATASPLIT_PATH)

Loading data from  /Users/kevinmoore/Git Repositories/dtu-02456-deep-learning-ebnerd/our_implementation/data with datasplit: ebnerd_small


## 4. Training the Model
- Initialize the NRMS model with preloaded hyperparameters and embeddings. 
- Train the model using the `train` directory in `DATASPLIT` bundle. 
- Early stopping is applied with a patience parameter of `3` to prevent overfitting.

In [4]:
# Loading Training and Validation data
train_loader, val_loader = prepare_training_data(
    hparams, PATH, DATASPLIT, article_mapping
)

Using 2 workers for dataloading
Using 2 workers for dataloading
 -> Train samples: 201537
 -> Validation samples: 32740


In [5]:
LOAD_FROM_CHECKPOINT = False

print("Training model with ", hparams)
device = torch.device("mps" if torch.backends.mps.is_available() else "cpu") # mps: running with Apple Silicon
model = NRMSModel(hparams, word_embeddings)

CHECKPOINT_DIR = save_hparams(device, hparams)

if not LOAD_FROM_CHECKPOINT: # Trains the model anew
	model = train_model(
			device,
			model,
			train_loader,
			val_loader,
			hparams,
			patience=3,
			checkpoint_dir=CHECKPOINT_DIR,
	)
else: 
		CHECKPOINT = "2024-12-20T19-41-58" # Change this to the checkpoint you want to load from
		
		import json
		CHECKPOINT_DIR = os.path.join(current_dir, "checkpoints/" + CHECKPOINT)
		INFO_FILE_PATH = os.path.join(CHECKPOINT_DIR, "info.json")

		with open(INFO_FILE_PATH, "r") as json_file:
					data = json.load(json_file)

		# Extract hyperparameters from JSON
		hparams_data = data.get("Hyperparameters", {})

		# Create an HParams instance with the loaded values
		hparams = HParams(**hparams_data)

		# Load the model from the checkpoint
		pth_file = "nrms_checkpoint_1.pth"
		checkpoint = torch.load(os.path.join(CHECKPOINT_DIR, pth_file))
		model = NRMSModel(hparams, word_embeddings=word_embeddings)
		model.load_state_dict(checkpoint['model_state_dict'])

Training model with  
 title_size: 30
 head_num: 20
 head_dim: 20
 attention_hidden_dim: 200
 dropout: 0.2
 batch_size: 32
 verbose: False
 data_fraction: 1
 sampling_nratio: 4
 history_size: 20
 epochs: 1
 learning_rate: 0.001
 transformer_model_name: facebookai/xlm-roberta-base
Training information saved to: checkpoints/2024-12-21T13-39-59


Validation: 100%|██████████| 1024/1024 [00:40<00:00, 25.58batch/s]


Epoch 1/1, Train Loss: 15.8500, Val Loss: 1.9656, Val AUC: 0.5559, Improvement from Previous Epoch: 0.5559
Checkpoint saved to: checkpoints/2024-12-21T13-39-59/nrms_checkpoint_1.pth


## 5. Evaluation using "Validation" directory
Using `MetricEvaluator` from `ebnerd-benchmark` repo

- **5.1** Loading the data

In [6]:
evaluation_loader = prepare_validation_data(
    hparams, PATH, DATASPLIT, article_mapping
)

Using 2 workers for dataloading
 -> Validation samples: 244647


- **5.2** Predict scores

In [7]:
scores, labels  = predict_scores(model, evaluation_loader, device)

Testing: 100%|██████████| 7646/7646 [08:06<00:00, 15.71batch/s]


- **5.3** Get Metrics using `MetricEvaluator`

In [8]:
metrics = MetricEvaluator(
		labels=labels,
    predictions=scores,
    metric_functions=[AucScore(), MrrScore(), NdcgScore(k=5), NdcgScore(k=10)],
)
metrics.evaluate()

AUC: 100%|███████████████████████████| 244647/244647 [00:05<00:00, 48426.97it/s]


<MetricEvaluator class>: 
 {
    "auc": 0.5589647606087678,
    "mrr": 0.3468401389415494,
    "ndcg@5": 0.3865613547918292,
    "ndcg@10": 0.4640687192683466
}

## 6. Final testing and writing submission file
Inspired from https://github.com/ebanalyse/ebnerd-benchmark/blob/main/examples/reproducibility_scripts/ebnerd_nrms.py

- Using data from `ebnerd_testset` directory
- Articles and embeddings loaded anew from above directory's `articles.parquet`
- Test data loaded from `test` subdirectory
- We split the testing data with and without beyond accurracy parameter similarly to the Github script

In [9]:
TESTSET_DIR = Path(os.path.join(PATH, "ebnerd_testset"))
TESTSET_PATH = Path(os.path.join(TESTSET_DIR, "test"))

print("Loading testing data from ", TESTSET_DIR)
article_mapping_test, word_embeddings_test = load_articles_and_embeddings(hparams, TESTSET_DIR)

df_test_wo_ba, df_test_w_ba, loader_wo_ba, loader_w_ba = prepare_test_data(
    hparams, TESTSET_PATH, article_mapping=article_mapping_test
)

Loading testing data from  /Users/kevinmoore/Git Repositories/dtu-02456-deep-learning-ebnerd/our_implementation/data/ebnerd_testset
Using 2 workers for dataloading
Using 2 workers for dataloading
 -> Testing samples: 13536710


- Run the trained NRMS model on the testing dataset and predict scores

In [10]:
EVALUATION_DIR = Path(os.path.join(CHECKPOINT_DIR, "evaluation"))
os.makedirs(EVALUATION_DIR, exist_ok=True)

# Without beyond accuracy
scores_wo_ba, labels_wo_ba  = predict_scores(model, loader_wo_ba, device)
df_pred_test_wo_ba = save_ranked_scores(df_test_wo_ba, scores_wo_ba, EVALUATION_DIR, "wo_ba")

# With beyond accuracy
scores_w_ba, labels_w_ba = predict_scores(model, loader_w_ba, device)
df_pred_test_w_ba = save_ranked_scores(df_test_w_ba, scores_w_ba, EVALUATION_DIR, "w_ba")

df_pred_test = pl.concat([df_pred_test_wo_ba, df_pred_test_w_ba])
df_test_predictions_parquet = df_pred_test.select(DEFAULT_IMPRESSION_ID_COL, "ranked_scores")
df_test_predictions_parquet.write_parquet(EVALUATION_DIR.joinpath("test_predictions.parquet"))

Testing: 100%|██████████| 6250/6250 [29:04<00:00,  3.58batch/s]


### Writing Submission File

In [11]:
write_submission_file(
    impression_ids=df_pred_test[DEFAULT_IMPRESSION_ID_COL],
    prediction_scores=df_pred_test["ranked_scores"],
    path=EVALUATION_DIR.joinpath("predictions.txt"),
    filename_zip=f"{model.__class__.__name__}-{SEED}-{DATASPLIT}.zip",
)

13536710it [13:33, 16645.26it/s]


Zipping checkpoints/2024-12-21T13-39-59/evaluation/predictions.txt to checkpoints/2024-12-21T13-39-59/evaluation/NRMSModel-42-ebnerd_small.zip
